# 1. Data preprocessing

Match tribe names from the trait data file with the equivalent names in the dplace phylogenetic trees

In [1]:
library(ape)

tree <- read.nexus('austronesian.trees', force.multi = FALSE)
tree.tribes <- tree$tree.65200000.305376.481665$tip.label

#tree <- read.nexus('austro_summary.tree', force.multi = FALSE)
#tree.tribes <- tree$tip.label#$tree.65200000.305376.481665$tip.label

df <- read.table('InitiationFromMike.csv',sep=',',header=TRUE)

#list of tribes (rows below Hawiian is junk for our purposes)
last <- which(df$Tribe == 'Hawaiian')
df <- df[1:last,]
tribe <- df$Tribe


## Phase one: match using pmatch
ii<-pmatch(tribe,tree.tribes)

#create tribe list matched with tree tips
tribe.lst <- tree.tribes[ii]
names(tribe.lst) <- tribe

# remaining unmatched  
tribe.um <- names(tribe.lst[is.na(tribe.lst)])
tree.um <- tree.tribes[!tree.tribes %in% tribe.lst[!is.na(tribe.lst)]]


## Phase two: match using grep
#### inspect the partial matches because looser srting match can yield false matches ####
grep.match <- lapply(tribe.um, grep,x=tree.um,value=TRUE)
grep.match <- lapply(grep.match, function(x) {x[1]})

#update tribe.lst
tribe.lst[tribe.um] <- as.character(grep.match)

# remaining unmatched  
tribe.um <- names(tribe.lst[is.na(tribe.lst)])
tree.um <- tree.tribes[!tree.tribes %in% tribe.lst[!is.na(tribe.lst)]]


#Phase three: match the remaning manually
manual.match <- c('Bajo','SubanunSindangan','TagbanwaKalamianCoronIsland_D','TobaBatak','Tetum',
            'HituAmbon','MaloSantaCruz','FutunaEast','UveaEast')

#update tribe.lst
tribe.lst[tribe.um] <- as.character(manual.match)
# remaining unmatched  
tribe.um <- names(tribe.lst[is.na(tribe.lst)])
tree.um <- tree.tribes[!tree.tribes %in% tribe.lst[!is.na(tribe.lst)]]

df$dplacenames <- as.character(tribe.lst)

# make name changes according to Mike's notes
tribe <- as.character(df$Tribe)

tribe[17] <- "Ida\'an"
tribe[25] <- "Minangkabau"
tribe[28] <- "Tae\'"
tribe[54] <- "Sa\'a"
tribe[59] <- "Ponapean"
tribe[65] <- "Tongan"

tribe[tribe == "Rotuma"] <- "Rotuman"
tribe[tribe == "Uvea East"] <- "Uvea"
tribe[tribe == "Toba Batak"] <- "Toba"
tribe[tribe == "Hitu Ambon"] <- "Hitu"
tribe[tribe == "East Futuna"] <- "East"
tribe[tribe == "Santa Cruz"] <- "Malo"
tribe[tribe == "Cham"] <- "Hainan"
tribe[tribe == "Atayal"] <- "Squliq"

df$Tribe <- tribe

write.table(df,'InitiationFromMike_dplacenames.csv',sep=',')

We then prune the Austronesian phylogenetic tree so it includes only tribes for which we have trait data. We also put trait data on initiation rites into the format required by BayesTraits

In [3]:
library('ape')
#library('phytools')

#load austronesian tree
tree <- read.nexus('austronesian.trees', force.multi = FALSE)

df <- read.table('InitiationFromMike_dplacenames.csv',sep=',',header=TRUE)
df<-df[df$Initiation.Rituals != 'No info',]
soc <- df$dplacenames

# tips for which we have no data
all.tips <- tree$tree.42500000.305341.333493$tip.label
remove <- all.tips[-match(soc, all.tips)]

# prune all 100 Austronesian Phylogenies
pruned.tree <- lapply(tree,drop.tip,tip=remove)
class(pruned.tree) <- 'multiPhylo'

write.nexus(pruned.tree, file = "pruned_multi.trees")

#prune the summary tree
sum.tree <- read.nexus('austro_summary.tree')
sum.tree <- drop.tip(sum.tree, tip=remove)
write.nexus(sum.tree, file = "pruned.tree")

ls <- as.character(df$Initiation.Rituals)
names(ls) <- df$dplacenames

ls[ls == 'Both'] = 'B'
ls[ls == 'None'] = 'N'
ls[ls == 'Male'] = 'M'
ls[ls == 'Female'] = 'F'

write.table(ls,'initiation.txt',sep='\t',quote=FALSE, col.names=FALSE)

plot traits on a fan phylogenetic tree of Austronesian languages

In [1]:
library('ape')
#library('phytools')
library('tidytree')

#trait <- read.table('initiation.txt',sep='\t',stringsAsFactors = FALSE,row.names = 1)
lookup <- read.csv('InitiationFromMike_dplacenames.csv',stringsAsFactors = FALSE,row.names = 1)#[row.names(trait),]
tree <- read.nexus('pruned.tree', force.multi = FALSE)
lookup$Tribe <- paste(rownames(lookup),lookup$Tribe)

row.names(lookup) <- lookup$dplacenames
lookup <- lookup[lookup$Initiation.Rituals != 'No info',]

sequential.tribe <- lookup$Tribe

lookup <- lookup[tree$tip.label,]

tree$tip.label <- lookup$Tribe

trait <- lookup$Initiation.Rituals

trait[trait == 'Both'] = 'Male and Female'

colors <- c("orange","purple",'black','skyblue')
labels <- c('Female','Male and Female','None','Male')

#pdf('phylotree.pdf',width=15,height=8) #horizontal tree settings
pdf('phylotree.pdf',width=15,height=12)
plot(tree,direction = 'upwards', label.offset=0.3,type='fan')

col.vec <- trait

for (i in seq(length(colors))) { col.vec[col.vec == labels[i]] = colors[i] }

#tt <- row.names(trait)[trait == 'Male']
#tiplabels(tt,col = colors[i], pch = 20,  adj = 0.5, cex = 2.5)
tiplabels(pch = 19, col = col.vec, adj = 0.5, cex = 2)

legend(6,1.5,legend = labels,fill = colors,xpd = T, cex = 1.1)

dev.off()


Attaching package: ‘tidytree’


The following object is masked from ‘package:stats’:

    filter




pdf 
  2

Run RJMCMC inference to discover the most likely modes of cultural evolution between the four initiation rites. Also infer the most likely initiation rite for the Austronesian ancestral state. 

We do this using a Bayesian phylogenetic inference engine called BayesTraits http://www.evolution.rdg.ac.uk/BayesTraitsV3.0.2/BayesTraitsV3.0.2.html   

Follow instructions from the website to install and then run our model using the following command in a Linux-style command line: *$ ./BayesTraitsV3 pruned_multi.trees initiation.txt < bayes_settings.txt*


Then print the transition probabilities between initiation rites and probabilities of ancestral states. We do this first across all evolution modes, then just for the most likely constrained mode of evolution.

This code also produces a figure illustrating the likely modes of evolution with ancestral state probabilities

Special thank you to Sam Passmore (University of Bristol) for his code to read Bayestraits into R for manipulation in R (https://rdrr.io/github/SamPassmore/excdr/)

In [8]:
library(diagram)
library(tidyr)
#cite (

read.bayestraits <- function(filename){
  con = file(filename, "r")
  i = 1
  while ( TRUE ) {
    line = readLines(con, n = 1)
    if (grepl("Iteration\t", line)|grepl("Tree\t", line)||grepl("Tree No\t", line)) {
      break
    }
    i = i + 1
  }
  close(con)

  read.table(filename, skip = i-1, sep = '\t',
             header = TRUE, check.names = FALSE, quote=NULL)
}
read.stones <- function(filename){
  con = file(filename, "r")
  i = 1
  while (TRUE) {
    line = readLines(con, n = 1)
    if(length(line) == 0){
      cat("no log marginal likelihood found")
      value = NA
      break
    }
    if(grepl("Log marginal likelihood:	", line)) {
      places = regexpr("-?[0-9.]+", line)
      value = as.numeric(substring(line, places[1]))
      break
    }
    i = i + 1
  }
  close(con)
  value
}


get_average_transions <- function(df) {
    
    means <- colMeans(df[,c('qBF', 'qBM', 'qBN', 'qFB', 'qFM', 'qFN', 'qMB', 'qMF', 'qMN', 'qNB', 'qNF', 'qNM',
     'Root P(B)', 'Root P(F)', 'Root P(M)', 'Root P(N)')])

    print('transisions and ancestral state probabilities across all models')
    mean_transitions <- sort(means)
    print(mean_transitions)
    
    return(mean_transitions)
    
}

filter_only_most_likely_model <- function(df) {
    freqModel <- sort(table(df['Model string']),decreasing=TRUE)
    model <- freqModel[1]

    print('probability of most popular model')
    print(as.double(model)/sum(freqModel))

    return(df[df['Model string'] == names(model),])
    
}

get_transition_matrix_and_root_probability <- function(mat_long) {
   
    #get long form matrix
    mat_long2 <- mat_long[!grepl('Root',names(mat_long))]
    
    

    trans_long <- data.frame(source = substr(names(mat_long2), start = 2, stop = 2),
         dest = substr(names(mat_long2), start = 3, stop = 3),
              trans = as.numeric(mat_long2))
    

    # wide form matrix
    M<-spread(trans_long, source, trans)
    rownames(M) <- M$dest
    M <- M[rownames(M)]
    

    #percentage probability that each state is the root 
    prob_root <- mat_long[grepl('Root',names(mat_long))]
    names(prob_root) <- substr(names(prob_root), start = 8, stop = 8)
    #round and match order from matrix M
    prob_root <- round(prob_root*100)[rownames(M)]

    # labels for each state
    rownames(M) <- c('Male\nand\nFemale','Female','Male','None') 
    
    return(list(M=M,prob_root=prob_root))
}

plot_and_save_transition_figure <- function(trans_overall,trans_bestmodel) {
    
    #get matrices of overall and bestmodel transitions and root states
    out<-get_transition_matrix_and_root_probability(trans_overall)
    M_overall <- out$M
    prob_root_overall <- out$prob_root

    out<-get_transition_matrix_and_root_probability(trans_bestmodel)
    M_bestmodel <- out$M
    prob_root_bestmodel <- out$prob_root


    #assign color based on probability the state is the root
    fc <- colorRampPalette(c("white", "darkred"))

    pdf('transition_plots.pdf',width=8,height=5)

    layout(matrix(c(rep(1,50),rep(2,50),rep(3,20)), 1, 120, byrow = TRUE))


    M <- M_overall
    prob_root<-prob_root_overall


    plotmat(M, curve=0.05,pos = c(2, 2),arr.lwd=M*20,relsize=1.25,cex.txt=0.,arr.width=M*1.5,arr.length = M*1.5,
           box.cex=1.4,box.col=fc(102)[prob_root+1])
    legend(x=0.1, y=0.82, '(a)', cex = 2.1,bty = "n")

    M <- M_bestmodel
    prob_root<-prob_root_bestmodel


    plotmat(M, curve=0.05,pos = c(2, 2),arr.lwd=M*20,relsize=1.25,cex.txt=0.,arr.width=M*1.5,arr.length = M*1.5,
           box.cex=1.4,box.col=fc(102)[prob_root+1])
    legend(x=0.1, y=0.82, '(b)', cex = 2.1,bty = "n")


    at<-c(1,seq(20,100,20))
    percent_string_convert <- function(i){paste(toString(i),'%',sep='')}
    labels <- lapply(at,percent_string_convert)

    image(y=1:102,z=t(1:102), col=fc(102), axes=F, main="Prob. state\nis root", cex.main=1.2,ylab='')
    axis(side = 2, las = 1, at=at,labels=labels)
    #axis(side = 2)

    dev.off()
}



df <- read.bayestraits('initiation.txt.Log.txt')

trans_overall <- get_average_transions(df)

df_bestmodel <- filter_only_most_likely_model(df)

trans_bestmodel <- get_average_transions(df_bestmodel)

plot_and_save_transition_figure(trans_overall,trans_bestmodel)

#print(c('log Marginal Lik', read.stones('initiation.txt.Stones.txt')))

[1] "transisions and ancestral state probabilities across all models"
       qNF  Root P(M)  Root P(N)        qBF  Root P(B)  Root P(F)        qNM 
0.07104862 0.18546455 0.18895949 0.20568635 0.27529080 0.35028512 0.45064074 
       qMN        qMF        qNB        qFB        qBN        qFN        qFM 
0.48192080 0.58103653 0.67493567 0.73874395 0.77693071 0.78674031 0.81378273 
       qMB        qBM 
0.86003927 0.96732691 
[1] "probability of most popular model"
[1] 0.04479167
[1] "transisions and ancestral state probabilities across all models"
      qBF       qFM       qMB       qMF       qMN       qNF       qNM Root P(B) 
0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 
Root P(M) Root P(N)       qBM       qBN       qFB       qFN       qNB Root P(F) 
0.0000000 0.0000000 0.3367061 0.3367061 0.3367061 0.3367061 0.3367061 1.0000000 


pdf 
  2